In [1]:
import csv
import collections

In [109]:
class Entry:
    def __init__(self, name, rooms):
        self.name = name.upper()
        self.rooms = rooms
        self.ratings = {}
        self.taken = set()

    def __str__(self):
        return '<{0} Entry>'.format(self.name)

    def __repr__(self):
        return self.__str__()

    def add_rating(self, freshman, rating):
        self.ratings[freshman.name] = rating

    def can_take(self, freshman):
        return self.rooms[freshman.gender] + self.rooms['U'] > 0 

    def take(self, freshman):
        self.taken.add(freshman)
        if self.rooms[freshman.gender] > 0:
            self.rooms[freshman.gender] -= 1
        else:
            self.rooms['U'] -= 1

    def add_to_round(self, freshman):
        self.current_round.add(freshman)

    def process_round(self):
        frosh = sorted(self.current_round, key=lambda f: (-self.ratings[f.name], len(Entries)))
        taken, dropped = set(), set()
        for freshman in frosh:
            if self.can_take(freshman):
                self.take(freshman)
                taken.add(freshman)
            else:
                dropped.add(freshman)

        return taken, dropped


In [158]:
class Freshman:
    def __init__(self, name, gender, rankings):
        self.name = name
        self.gender = gender
        self.rankings = rankings

    def __str__(self):
        return '<Freshman: {0}>'.format(self.name)

    def __repr__(self):
        return self.__str__()

    def favorite_entry(self):
        if min(self.rankings.values()) == float('inf'):
            raise Exception(self.rankings)
        Min = 10000
        Entry = None 
        for key in self.rankings:
            if self.rankings[key] < Min: 
                Min = self.rankings[key]
                Entry = key 
        return Entry

    def rejected_by(self, entry):
        self.rankings[entry.name] = float('inf')


#Imports

For each entry, create an instance of Entry and put it in Entries

In [111]:
def add_entries(entry_csv):
    with open(entry_csv) as f:
        for line in csv.reader(f):
            name = line[0].strip()
            m, f, u = map(int, line[1:])
            rooms = {
                'M': m,
                'F': f,
                'U': u,
            }
            entry = Entry(name, rooms)
            Entries[name] = entry

For each freshman, create an instance of Freshman and put it in Frosh

In [112]:
def frosh_prefs(frosh_prefs):
    with open(frosh_prefs) as f:
        for line in csv.reader(f):
            name = line[0].strip()
            gender = line[2].strip()
            a, b, c, d, e, f, g, h, j = map(int, line[3:])
            rankings = dict(zip(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'], [a, b, c, d, e, f, g, h, j]))

            if set(rankings.values()) != set(range(1, 10)):
                raise Exception('Unusable preferences: {0}'.format(name))
            freshman = Freshman(name, gender, rankings)
            Frosh[name] = freshman

For each frosh, add entry ratings. If improperly formatted, will return infinity

In [113]:
def entry_prefs(entry_prefs):
    with open(entry_prefs) as f:
        for line in csv.reader(f):
            name = line[0].strip()
            s = ''
            for char in name: 
                if char == ' ':
                    s += ';'
                else:
                    s+= char
            freshman = Frosh[s]
            if len(line[1:]) == 9:
                a, b, c, d, e, f, g, h, j = map(int, line[1:])
                for entry_name, rating in zip(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'], [a, b, c, d, e, f, g, h, j]):
                    entry = Entries[entry_name]
                    entry.add_rating(freshman, rating)
            else: 
                del Frosh[freshman.name]

#If entry gave a 6 and frosh put them in top 3, place automatically 

In [114]:
def greasing(Entries, Frosh):
    greasing = {}
    for entry in Entries.values():
        for f_name, rating in entry.ratings.items():
            freshman = Frosh[f_name]
            ranking = freshman.rankings[entry.name]
            if rating == 6 and ranking <= 3:
                if freshman in greasing:
                    greasy_entry = greasing[freshman]
                    if freshman.rankings[greasy_entry.name] < ranking: # frosh likes greasy better
                        greasing[freshman] = entry
                else:
                    greasing[freshman] = entry
    return greasing

In [115]:
def grease_placement(greasing, Entries, Frosh):
    for freshman, entry in greasing.items():
        if entry.can_take(freshman):
            entry.take(freshman)
            del Frosh[freshman.name]


#Marriage Algorithm

In [174]:
def getMarried(Entries, Frosh):
    while len(Frosh) > 0:  
        for entry in Entries.values(): 
            entry.current_round = set()
        for freshman in Frosh.values(): 
            favorite_entry = freshman.favorite_entry()
            Entries[favorite_entry].current_round.add(freshman)
        for entry in Entries.values():
            taken, dropped = entry.process_round()
            for freshman in taken: 
                del Frosh[freshman.name]
            for freshman in dropped: 
                freshman.rejected_by(entry)         
    return Entries 
    

#Let's do it!!

In [175]:
Entries = {}
Frosh = {}

In [176]:
add_entries('entries.csv')
frosh_prefs('finalfroshprefs.csv')
entry_prefs('entryprefs.csv')

In [177]:
g = greasing(Entries, Frosh)
grease_placement(g, Entries, Frosh)
getMarried(Entries, Frosh)

{'A': <A Entry>,
 'B': <B Entry>,
 'C': <C Entry>,
 'D': <D Entry>,
 'E': <E Entry>,
 'F': <F Entry>,
 'G': <G Entry>,
 'H': <H Entry>,
 'J': <J Entry>}

In [181]:
## CHECK 1: ALL FROSH TAKEN 
print "CHECK 1: ", len(Frosh) == 0
## CHeck 2: No FROSH PLACED TWICE
froshies = []
for entry in Entries.values():
    for frosh in entry.taken:
        if frosh not in froshies: 
            froshies.append(frosh)
        else:
            print "CHECK 2: ", False
print "CHECK 2: ",True

CHECK 1:  True
CHECK 2:  True


In [184]:
for entry in Entries.values(): 
    print entry.name 
    for freshman in entry.taken: 
        print freshman.name
    print "\n"

A
Jessica;Pointing
Clare;Farley
Abraham;Gertler
Cristina;Martinez-Acha
Vitoria;Moreno-Costa
Luke;Weisenbach
David;Klee
Yoeal;Efrem
Mariya;Layurova
Shu-Yun(Gina);Liu
Alexandra;Angelo
An;Liu
Richard;Yip
Luana;Lara
Josef;Hazi
Ekin;Karasan
Bryan;Lilley


C
Matthew;McClain
Qingqi;Zeng
Varun;Mohan
Kevin;Zhu


B
Elliot;Owen
Alasdair;Hastewell
James;Mawdsley
Vickie;Wang
Andrea;Herman
Isabella;Pecorari
Marayna;Martinez
Michael;Janner
Katherine;Paseman


E
Devany;West
David;Vaccaro
Megan;Chao
Aaron;Zeng
Kimberly;Dauber
Isac;Hernandez


D
Natalie;Nicolas
Sicong;Shen
Ji-Hyun;Min
Hiram;Moncivais
Caroline;Liu
Joyce;Chen
Richard;Ibekwe
Donovan;Sienkiewicz
Michelle;Huang
Jae-Hyun;Kim
Stacy;Ho
Barry;McNamara
Kevin;Liu
Reid;Akana
Jennifer;Switzer
Leigh;Braswell
Joseph;Lin


G
Oluwafemi;Oladipupo
Jing-Xian;Wang
Jesse;Gibson
Kolby;Danner
Katie;Dunn
Ty;Ingram
Trang;Luu
Matthew;Khoury
Zachary;Hall
Lisa;Deng
Mustafa;Ben
Caspar;Stinn
Alan;Alahmad
Ethan;Witt


F
Mohannad;Abunassar
Elizabeth;Strand
William;Harv